In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from glob import glob
from pymystem3 import Mystem
from many_stop_words import get_stop_words

In [2]:
code = pd.read_csv('data/criminal_code.csv')

In [3]:
df = pd.DataFrame()
for path in glob('data/spiders_data/*.jl'):
    df = pd.concat([df, pd.read_json(path, lines=True)], ignore_index=True)

In [4]:
df['title'] = df['title'].map(lambda x: x[0])
df['body']= df['body'].map(lambda x: x[0])

In [5]:
def parse_table(table_body):
    data = dict()
    for line in BeautifulSoup(table_body, 'lxml').find_all('tr'):
        data[line.find('td').get_text()] = line.find('th').get_text()
    return data

Выкидываем записи с пустой таблицей метаданных:

In [6]:
df = df.drop(df[df['table'].map(lambda x: len(x)==0)].index)

Парсим таблицу метаданных:

In [7]:
df['table'] = df['table'].map(lambda x: parse_table(x[0]))
df = df.join(df['table'].apply(pd.Series)).drop('table', axis=1)

Разделяем судебные решения словами `установил` <содержательная часть> `приговорил` <решение суда>

In [8]:
df_pr = df[df['body'].map(lambda text: any([phrase in re.sub(r"\s+", "", text.lower(), flags=re.UNICODE) for phrase in ['судприговорил', 'приговорил']]))].reset_index()

In [9]:
df_pr['body'] = df_pr['body'].map(lambda text: re.sub('у\s?с\s?т\s?а\s?н\s?о\s?в\s?и\s?л', 'установил', text.lower()))

In [10]:
df_pr = df_pr[df_pr.body.str.split('установил').map(len) == 2]

In [11]:
df_pr['body'] = df_pr['body'].str.split('установил')

In [12]:
df_pr['second'] = df_pr['body'].map(lambda x: re.sub('п\s?р\s?и\s?г\s?о\s?в\s?о\s?р\s?и\s?л', 'приговорил', x[1]))

In [13]:
df_pr = df_pr[df_pr['second'].str.split('приговорил').map(len) == 2]

In [14]:
df_pr['case'], df_pr['result'] = df_pr['second'].map(lambda text: text.replace(':','').split('приговорил')).str

Из решения суда выделяем номера статей кодекса:

In [15]:
pattern = re.compile(r'(статьей|ст\.|статьи)\s*(\d+\.?\d*)')
df_pr['labels'] = df_pr['result'].map(lambda text: list(map(lambda x: float(x[1]), re.findall(pattern, text))))

Оставляем только те статьи кодекса, которые мы знаем:

In [16]:
df_pr['labels'] = df_pr['labels'].map(lambda arr: list(filter(lambda x: x in code['number'].tolist(), arr)))

In [17]:
df_pr = df_pr[df_pr['labels'].map(len) != 0]

In [18]:
df_pr['case'] = df_pr['case'].str.strip()

In [19]:
df_pr = df_pr[['title', 'Категория', 'Дата', 'Регион', 'Суд', 'Судья', 'second', 'case', 'result', 'labels']].rename(columns={
    'Категория':'category',
    'Дата':'date',
    'Регион':'region',
    'Суд':'court',
    'Судья': 'judge',
    'second': 'description',
    'case': 'reasoning'
})

In [20]:
print('Количество судебных решений ', df_pr.shape[0])

Количество судебных решений  9533


In [21]:
df_pr.to_csv('data/judicial_orders.csv', index=False)

In [22]:
m = Mystem(entire_input=False)
ru_stopwords= get_stop_words('ru')

In [23]:
df_pr['lemmas'] = df_pr['reasoning'].map(lambda text: list(filter(lambda word: word not in ru_stopwords and len(word) > 3, m.lemmatize(text))))
df_pr[['title', 'lemmas', 'labels']].to_csv('data/judicial_orders_lemmas.csv', index=False)